# Model Building

In [1]:
import pandas as pd
import numpy as np

In [2]:

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

In [3]:
df=pd.read_csv("data/bhpp_model.csv")
df.head(1)

,Unnamed: 0,size,total_sqft,bath,price,bhk,price_per_sqft,1st Block Jayanagar,1st Phase JP Nagar,2nd Phase Judicial Layout,...,Vijayanagar,Vishveshwarya Layout,Vishwapriya Layout,Vittasandra,Whitefield,Yelachenahalli,Yelahanka,Yelahanka New Town,Yelenahalli,Yeshwanthpur
0,0,4 BHK,2850.0,4.0,428.0,4,15017.54386,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [4]:
X = df.drop(["size",'Unnamed: 0',],axis='columns')
X.head()

,total_sqft,bath,price,bhk,price_per_sqft,1st Block Jayanagar,1st Phase JP Nagar,2nd Phase Judicial Layout,2nd Stage Nagarbhavi,5th Block Hbr Layout,...,Vijayanagar,Vishveshwarya Layout,Vishwapriya Layout,Vittasandra,Whitefield,Yelachenahalli,Yelahanka,Yelahanka New Town,Yelenahalli,Yeshwanthpur
0,2850.0,4.0,428.0,4,15017.543860,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1630.0,3.0,194.0,3,11901.840491,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,1875.0,2.0,235.0,3,12533.333333,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,1200.0,2.0,130.0,3,10833.333333,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,1235.0,2.0,148.0,2,11983.805668,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [5]:
df.to_csv('data/final_bhpp_model.csv')

In [14]:
y = df.price
y.head()

0    428.0
1    194.0
2    235.0
3    130.0
4    148.0
Name: price, dtype: float64

In [15]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42)


In [16]:
lr_clf = LinearRegression()
lr_clf.fit(X_train,y_train)
lr_clf.score(X_test,y_test)

0.8936824572836449

In [17]:
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
cross_val_score(LinearRegression(), X, y, cv=cv)

array([0.93679418, 0.85126017, 0.92708597, 0.9297952 , 0.93501809])

In [18]:
def find_best_model_using_gridsearchcv(X, y):
    algos = {
        'linear_regression': {
            'model': LinearRegression(),
            'params': {
                'fit_intercept': [True, False],  # Add appropriate parameters here
                'copy_X': [True, False],
                'n_jobs': [-1, 1, 2, 4],  # Just an example, you can customize
                'positive': [True, False]  # Depending on your requirements
            }
        },
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1, 2],
                'selection': ['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion': ['mse', 'friedman_mse'],
                'splitter': ['best', 'random']
            }
        }
    }
    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    for algo_name, config in algos.items():
        gs = GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False)
        gs.fit(X, y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores, columns=['model', 'best_score', 'best_params'])


In [19]:
# comparing all models

find_best_model_using_gridsearchcv(X,y)

c:\projects\BHPP\bhpp_env\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
10 fits failed out of a total of 20.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "c:\projects\BHPP\bhpp_env\lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\projects\BHPP\bhpp_env\lib\site-packages\sklearn\base.py", line 1145, in wrapper
    estimator._validate_params()
  File "c:\projects\BHPP\bhpp_env\lib\site-packages\sklearn\base.py", line 638, in _validate_params
    validate_parameter_constraints(
  File "c:\projects\BHPP\bhpp_env\lib\site-pack

,model,best_score,best_params
0,linear_regression,0.915991,"{'copy_X': True, 'fit_intercept': True, 'n_job..."
1,lasso,0.902707,"{'alpha': 1, 'selection': 'random'}"
2,decision_tree,0.930741,"{'criterion': 'friedman_mse', 'splitter': 'ran..."


In [20]:
# function for model testing
def predict_price(location,sqft,bath,bhk):    
    loc_index = np.where(X.columns==location)[0][0]

    x = np.zeros(len(X.columns))
    x[0] = sqft
    x[1] = bath
    x[2] = bhk
    if loc_index >= 0:
        x[loc_index] = 1

    return lr_clf.predict([x])[0]

Here predicting the house prices based on Location, Size, Bathroom, and BHK

In [21]:
predict_price('1st Phase JP Nagar',1000, 2, 2)

c:\projects\BHPP\bhpp_env\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


-44.746680089763444

In [22]:
predict_price('1st Phase JP Nagar',2000, 3, 3)

c:\projects\BHPP\bhpp_env\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


18.267501808232737